# <font color="blue">Workshop : Hands-on Data PreProcessing using Python</font>

- by Shivam Bansal | Feb 20, 2019 | NUS BAC

<hr>

### Contents 

1. Data Pre-Processing     
2. PreProcessing - Numerical Data  
3. PreProcessing - Text Data  


<br>
<hr> 

### <font color="blue">1. What is Data PreProcessing ?</font> 

#### Life Cycle of a Data Science Project 

A typical data science project lifecycle consists of 7 main steps : 

1. Business Use Case Understanding   
2. Relevant Data Collection   
3. Data PreProcessing   
4. Exploration of Data   
5. New Features Engineering    
6. Modelling  - Descriptive or Predictive    
7. Insights Extraction   

<img src="http://sudeep.co/images/post_images/2018-02-09-Understanding-the-Data-Science-Lifecycle/chart.png" width=400>

<br>

#### Effort and Time Spend 

<img src="https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fblogs-images.forbes.com%2Fgilpress%2Ffiles%2F2016%2F03%2FTime-1200x511.jpg">

<br><br><br><hr>

### <font color="blue">What is Data Preprocessing ?</font> 

Real-world data is often incomplete, inconsistent, and/or lacking in certain behaviors or trends, and is likely to contain many errors.      

<br>
For Example: 
<br>

<br>
<img src="https://i.imgur.com/qSDfmu5.png" width=600>
<br>

Another Example:   

<br>
<img src="https://i.imgur.com/xZl4Wtt.png" width=500>
<br>

Another Example:   

<br>
<img src="https://powerspreadsheets.com/wp-content/uploads/excel-date-format-dddd-mmmm-dd-yyyy.jpg" width=500>
<br>


<b>Data preprocessing is a systematic process of cleaning a dataset so that a raw dataset is converted into a standardized one.</b>   

**Why DataPreProcessing ?** 
- Quality of your inputs decide the quality of your output   
-- Uncleaned Data can lead to a biased analysis or a biased model  
-- Uncleaned Data can produce to noisy and wrong results  
- Without preprocessing, the problem size increases  
-- model's can become very complex   
-- more training time  
-- high chances of non-convergence  
-- Without preprocessing, dimentionality of data space is very huge, more computation is required   
- Better Interpretability of results and model outputs   
- Can lead to improvement in accuracy  


<br><hr> 

### <font color="blue">2 : PreProcessing Numerical Data</font> 

Dataset : House Price Prediction
Label : House Price  
Features : Attributes of a House  

First load the dataset in a pandas dataframe 

In [ ]:
import pandas as pd 
data = pd.read_csv("data/house_prices.csv")
data.head()

Ideally, the first step is to understand the data and different variables. It involves: 
- identify the target variable and the independent variables  
- identfiy the continuous and categorical variables  
- identify the datatypes of different variables   

### 2.1 Missing Values Treatment   

1. Identify the columns with missing values  
2. Individually handle the columns  
    -- Drop the columns having large missing values  
    -- Impute the continuous columns by stratified mean / median  
    -- Impute the categorical columns by stratified mode  
    -- Predict the missing values using regression models  

In [ ]:
## 1. identify the columns with missing values 
def get_missing_columns():
    missing_df = data.isna().sum(axis=0).to_frame()
    missing_df = missing_df.reset_index().rename(columns = {0:"missing_count"})
    missing_df = missing_df.sort_values("missing_count", ascending = False)
    missing_df = missing_df[missing_df["missing_count"] > 0]
    return missing_df

get_missing_columns()

In [ ]:
## 1. Drop Columns 
drop_cols = ["PoolQC", "MiscFeature", "Alley", "Fence",  "FireplaceQu"]
data = data.drop(drop_cols, axis = 1)
get_missing_columns()

In [ ]:
## 2. Stratified Replacement
data["LotFrontage"]

lookup = {}
for i,row in data.groupby("Neighborhood").agg({"LotFrontage" : "median"}).reset_index().iterrows():
    key = row["Neighborhood"]
    value = row["LotFrontage"]
    lookup[key] = value  
lookup

In [ ]:
import numpy as np 
data["LotFrontage"] = data.apply(lambda x : lookup[x["Neighborhood"]] if pd.isnull(x["LotFrontage"]) else x["LotFrontage"]  , axis = 1)

In [ ]:
data["LotFrontage"]
get_missing_columns()

In [ ]:
cols_to_fix = [c for c in data.columns if "Garage" in c]

for c in cols_to_fix:
    if data[c].dtype == "object":
        data[c] = data[c].fillna("NA")
    else:
        data[c] = data[c].fillna(0.0)

In [ ]:
data[data["GarageType"].isna()][cols_to_fix].head(10)

In [ ]:
get_missing_columns()

In [ ]:
cols_to_fix += [c for c in data.columns if "Bsmt" in c]
cols_to_fix += [c for c in data.columns if "Mas" in c]

for c in cols_to_fix:
    if data[c].dtype == "object":
        data[c] = data[c].fillna("NA")
    else:
        data[c] = data[c].fillna(0.0)

get_missing_columns()

In [ ]:
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
get_missing_columns()

In [ ]:
data.head()

### 2.2 Outliers Treatment

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots()
ax.scatter(x = data['GrLivArea'], y = data['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show();

In [ ]:
#Deleting outliers
data = data.drop(data[(data['GrLivArea']>4000) & (data['SalePrice']<300000)].index)

#Check the graphic again
fig, ax = plt.subplots()
ax.scatter(data['GrLivArea'], data['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show();

In [ ]:
fig, ax = plt.subplots()
ax.scatter(data['GarageArea'], data['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GarageArea', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(data['YearBuilt'], data['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('YearBuilt', fontsize=13)
plt.show()

### 2.3 Removal of Least Informative Columns

-- Almost Constant Columns (Less Variation)  
-- Duplicate Columns  

In [ ]:
var_df = data.var().to_frame().reset_index().rename(columns = {"index" : "column", 0:"variance"})
var_df = var_df.sort_values("variance", ascending = True)
var_df

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in data.columns:
    if data[col].dtype == "object":
        data[col] = le.fit_transform(data[col].values)

In [ ]:
var_df = data.var().to_frame().reset_index().rename(columns = {"index" : "column", 0:"variance"})
var_df = var_df.sort_values("variance", ascending = True)
var_df

In [ ]:
data = data.drop(['Utilities', "Street", "Id"], axis=1)

In [ ]:
## Duplicate Rows 
print (data.shape)
data["a_duplicate_columns1"] = data["MasVnrArea"]
data["a_duplicate_columns2"] = data["MasVnrArea"]
data["a_duplicate_columns3"] = data["MasVnrArea"]
data["a_duplicate_columns4"] = data["MasVnrArea"]
data["a_duplicate_columns5"] = data["MasVnrArea"]
data["a_duplicate_columns6"] = data["MasVnrArea"]
print (data.shape)

# data.drop_duplicates()
data = data.T.drop_duplicates().T
print (data.shape)

In [ ]:
data.head()

In [ ]:
## Normalization 

from sklearn.preprocessing import MinMaxScaler
numerical_columns = [column for column in data.columns if data[column].dtype != "object"]

sc = MinMaxScaler()
data[numerical_columns] = sc.fit_transform(data[numerical_columns].values)
data.head()

Recap  
-- Missing Value Treatment  
-- Outlier Treatment  
-- Redundant Columns Removal  
-- Normalization  


## 3 : Handson PreProcessing of Text Data 

1. Stopwords Removal
2. Special Characters Removal
3. Text Normalization (LowerCasing, Lemmtization, Stemming)
4. Regular Expressions based Removal  - hashtags, urls, mentions  

In [ ]:
textdf = pd.read_csv("data/finance_tweets.csv", error_bad_lines = False)
textdf.head()

In [ ]:
import string 
punc = string.punctuation
punc

In [ ]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
custom = ["@", "#", "www", "http"]

In [ ]:
def clean_my_text(text):
    
    ## lower casing 
    text = text.lower()
    # PLAYING, Playing, playing >>> playing 
    
    ## Stopwords Removal
    words = text.split()
    text1 = ""
    for word in words:
        if word not in stopwords:
            text1 += word 
            text1 += " "

    ## unnecessary noise removal
    words = text1.split()
    text2 = ""
    for word in words:
        if any(word.startswith(x) for x in custom):
            continue
        text2 += word 
        text2 += " "
    
    ## Punctuations Removal
    cleaned_text = "".join([char for char in text2 if char not in punc])
    
    ## Remove extra space
    cleaned_text 
    return cleaned_text
    
clean_my_text("""Today We are learning DATA PrEProcessing !!!. Hey??? We are learning at 
              Business - Analytics Center !!!. #datascience @Prof help us. Sign up @ https://msba.com . 12 23 405 2019 """)

In [ ]:
textdf["cleaned"] = textdf["text"].apply(clean_my_text)
textdf[["text", "cleaned"]][:100]